In [1]:
import requests
from bs4 import BeautifulSoup
import re

import concurrent.futures

In [2]:
with open('furniture stores pages.csv', 'r') as f:
    lines = f.readlines()
    urls = lines[1:]

urls = list(map(lambda x : re.sub(r'\n', '', x), urls))

In [3]:
response = requests.get(urls[0])
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
%%time
def scrape_product_info(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
    
        # Extract product information based on the HTML structure of the websites
        try:
            title = soup.find('title').text.strip()
            if "404 Not Found" in title:
                title = ""
        except:
            title = ""
        try:
            description = soup.find('meta', {'name': 'description'})['content'].strip()
            if "404 Not Found" in description:
                description = ""
        except:
            description = ""

        # try:
        #     paragraphs = [p.get_text() for p in soup.find_all('p') ]
        #     paragraphs = [p.strip() for p in paragraphs if 20 < len(p)]
            
        #     if "404 Not Found" in paragraphs:
        #         paragraphs = ""
        # except:
        #     paragraphs = ""

        response = [title] + [description] #+ paragraphs
    except:
        response = []

    # You can add more logic to extract other relevant information

    return response

def scrape_product_info_parallel(urls, max_workers=None):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Use map to apply the square_number function to each element of the list
        squared_numbers = list(executor.map(scrape_product_info, urls))
    return squared_numbers

product_info = scrape_product_info_parallel(urls)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


CPU times: user 3min 14s, sys: 31.6 s, total: 3min 46s
Wall time: 5min 57s


In [5]:
data = list()
_ = [data.extend(l) for l in product_info]

In [2]:
import joblib

In [4]:
joblib.dump(data, 'scraped_data_2.joblib', compress=('zlib', 3))

NameError: name 'data' is not defined

In [3]:
data = joblib.load('scraped_data_2.joblib')

In [71]:
data = list(map(lambda text: (re.sub(r'\n|\xa0|\\x[0-9a-fA-F]+', ' ', text),), product_info))

In [48]:
import sparknlp

spark = sparknlp.start()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cabe8ee6-9b44-47a6-8de9-cf27b73448cd;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.3.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.16.0 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failur

23/12/06 08:26:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [73]:
from pyspark.sql import SparkSession
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import RegexTokenizer

from pyspark.ml import Pipeline

# Sample DataFrame
# data = [("Hamar Plant Stand - Ash\n– The Modern; The Hamar Plant Stand is a wonderful accessory that holds at least three pot plants or other objects such as vases, sculptural objects or lamps. Use it to fill a corner or that tricky spot, bring height and greenery to your room.\xa0\xa0 Dimensions: 65cm at widest point x 60cm (D) x 92cm (H)Each platform has 28cm (W) x 28cm",)]

columns = ["text"]
df = spark.createDataFrame(data, columns)

# Define Spark NLP pipeline
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = RegexTokenizer().setInputCols(["document"]).setOutputCol("tokens").setPattern(r'\W+')
finisher = Finisher().setInputCols(["tokens"]).setOutputCols(["cleaned_text"]).setOutputAsArray(True)

# Assemble Spark NLP pipeline
pipeline = Pipeline(stages=[document_assembler, tokenizer, finisher])

# Fit the pipeline
model = pipeline.fit(df)

# Transform the DataFrame
result = model.transform(df)

# Show the result
result.select("cleaned_text").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/annotation/text/english/named-entity-recognition/ZeroShot_NER.ipynb)

# Named Entity Recognition with ZeroShotNer

## Colab Setup

In [2]:
# !pip install -q pyspark==3.3.0  spark-nlp==4.3.1

In [4]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e6e0af79-33a9-4eba-b545-d7f193b9c6b2;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.4 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failur

Spark NLP version:  5.1.4
Apache Spark version:  3.4.0


# Zero-shot Named Entity Recognition

`Zero-shot` is a new inference paradigm which allows us to use a model for prediction without any previous training step.

For doing that, several examples (_hypotheses_) are provided and sent to the Language model, which will use `NLI (Natural Language Inference)` to check if the any information found in the text matches the examples (confirm the hypotheses).

NLI usually works by trying to _confirm or reject an hypotheses_. The _hypotheses_ are the `prompts` or examples we are going to provide. If any piece of information confirm the constructed hypotheses (answer the examples we are given), then the hypotheses is confirmed and the Zero-shot is triggered.

Let's see it  in action.


In [5]:
spark.conf.set("spark.jsl.settings.pretrained.cache_folder","cache_pretrained")

In [6]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sen = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# sparktokenizer = Tokenizer()\
#     .setInputCols("sentence")\
#     .setOutputCol("token")

sparktokenizer = RegexTokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")\
    .setPattern(r'\W+')

 #"roberta_base_qa_squad2")\
zero_shot_ner = ZeroShotNerModel.pretrained("roberta_base_qa_squad2") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("zero_shot_ner")\
    .setEntityDefinitions(
        {
            "DATE": ['When was the company acquisition?', 'When was the company purchase agreement?'],
            "ORG": ["What's the brand's name?"],
            "PRODUCT": [
                    "What do you call a small, usually decorative table placed beside a bed?",
                    "What is a piece of furniture designed for sitting, typically with a backrest and armrests?",
                    "In a living room, what holds a television and media components?",
                    "What is a long, narrow table often placed against a wall or behind a sofa?",
                    "What is a piece of furniture with doors and shelves, used for storing dishes and linens?",
                    "What do you use for supporting a mattress for sleeping?",
                    "What is a small, low table placed in front of a sofa for holding drinks and books?",
                    "What is a tall, narrow piece of furniture with shelves for storing shoes or other items?",
                    "In a bedroom, what has drawers and a flat surface, used for storing clothes?",
                    "What is a large, comfortable piece of furniture for sleeping?",
                    "What do you call a piece of furniture with a mirror for applying makeup and storing toiletries?",
                    "What is a round or oval piece of furniture often used for dining?",
                    "In a dining room, what is a long, often rectangular piece of furniture for seating multiple people?",
                    "What is a small, usually decorative table often placed beside a sofa or chair?",
                    "What is a piece of furniture with a flat top and legs, often used for eating or working?",
                    "What is a long, cushioned seat often found in living rooms and bedrooms?",
                    "What is a piece of furniture with a flat surface and legs, often used for writing or working?",
                    "In a kitchen, what furniture item is used for preparing and serving meals?",
                    "What is a piece of furniture with a cushioned seat and a backrest, often found in dining rooms?",
                    "What is a piece of furniture with drawers and a flat surface, used for writing or working?",
                ],
            "MATERIAL": ["What's it made of?", "Which materials are used to make the product?", "What's the product manufactured of?"],
            "O": ["May i have a word that is not part of the MATERIAL, PRODUCT, ORG, DATE category?"]
        })

nerconverter = NerConverter()\
  .setInputCols(["sentence", "token", "zero_shot_ner"])\
  .setOutputCol("ner_chunk")

roberta_base_qa_squad2 download started this may take some time.
Approximate size to download 442.6 MB
[ | ]roberta_base_qa_squad2 download started this may take some time.
Approximate size to download 442.6 MB
Download done! Loading the resource.
[ | ]

2023-12-07 16:43:24.948447: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]


In [7]:
pipeline = None

In [8]:
pipeline =  Pipeline(stages=[
  documentAssembler,
  sen,
  sparktokenizer,
  zero_shot_ner,
  nerconverter
    ]
)

In [9]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [10]:
n_partitions = 1000

# spark.conf.set("spark.sql.shuffle.partitions", n_partitions)
# df = df.repartition(n_partitions)

In [11]:
from pyspark.sql.types import StructType,StructField, StringType
sample_text = data

p_model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

score_df = spark.createDataFrame(sample_text, StringType()).toDF("text")
score_df = score_df.repartition(n_partitions)

res = p_model.transform(score_df)

In [10]:
import pandas as pd
res_df=res.toPandas()

In [ ]:
%time
from pyspark.sql import functions as F

res_df = res.repartition(n_partitions).select(F.explode(F.arrays_zip(res.ner_chunk.result, res.ner_chunk.begin, res.ner_chunk.end, res.ner_chunk.metadata)).alias("cols")) \
   .select(F.expr("cols['0']").alias("chunk"),
           F.expr("cols['3']['entity']").alias("ner_label")).toPandas()
   # .show(truncate=False)
# .filter("ner_label!='O'")\

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.63 µs


In [ ]:
res_df.head(50)